In [ ]:
import os
import re
import csv
import json
import math
import openai
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from datetime import datetime
from collections import defaultdict, Counter
from matplotlib.colors import ListedColormap

In [ ]:
### Configurations

TIME_LIMIT = 30 # seconds

DEFAULT_SYS_MSG = "You are a helpful assistant."
SYS_MSG = DEFAULT_SYS_MSG

MODEL = 'turbo'
USE_CHAT = True

def switch_model(model):
    if model == 'turbo': model = 'gpt-3.5-turbo-0301'
    if model == 'gpt3': model = 'text-davinci-003'
    if model == 'gpt4': model = 'gpt-4-0314'

    global MODEL, USE_CHAT
    MODEL = model
    if model == 'gpt-4-0314':
        openai.api_key = 'YOUR_API_KEY'
    else: 
        openai.api_key = 'YOUR_API_KEY'

    if 'text' in model:
        USE_CHAT = False
    else:
        USE_CHAT = True

switch_model('turbo')

In [ ]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


In [ ]:
def get_response(
    messages: list = [],
    model=None,
    use_chat=None,
    **kwargs
) -> dict:
    if model is None: model = MODEL
    if use_chat is None: use_chat = USE_CHAT
    # print('model:', model, MODEL)
    with time_limit(TIME_LIMIT):
        if use_chat:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                **kwargs
            ).to_dict_recursive()
        else:
            prompt = '\n\n'.join([x['content'] for x in messages]) + '\n\n'
            response = openai.Completion.create(
                model=model,
                prompt=prompt,
                max_tokens=100,
                **kwargs
            ).to_dict_recursive()

    return response

def update_messages(
    messages, 
    responses, 
    prompt,
    **kwargs
):
    messages.append(
        {"role": "user", "content": prompt}
    )
    response = get_response(messages, **kwargs)
    responses.append(response)
    messages.append(
        response['choices'][0]['message'] if USE_CHAT else {
            'role': 'assistant',
            'content': response['choices'][0]['text']
        }
    )

In [ ]:
def run_one_session(
    prompts=[], 
    n_instances=30, 
    print_except=True,
    orders=None,
    tqdm_silent=False,
    **kwargs
):
    # global records
    records = {
        'messages' : [],
        'responses' : [],
    }
    if orders is None:
        orders = [list(range(len(prompts)))] * n_instances
    with tqdm(total=n_instances, disable=tqdm_silent) as pbar:
        i = 0
        while i < n_instances:
        # while pbar.n < n_instances:
            try:
                responses = []
                messages = [
                    {"role": "system", "content": SYS_MSG},
                ]
                for prompt_id in orders[i]:
                # for prompt_id in orders[pbar.n]:
                    prompt = prompts[prompt_id]
                    update_messages(
                        messages, 
                        responses, 
                        prompt,
                        **kwargs
                    )
                records['messages'].append(messages)
                records['responses'].append(responses)

                i += 1
                # pbar.update(1)
                
            except Exception as e: 
                if print_except: print(e)
                continue
    return records

In [ ]:
def extract_chatgpt(
    message, 
    extractor,
    print_except=True,
    type=float,
):
    prompts = [extractor(message)]
    records = run_one_session(
        prompts, 
        n_instances=1, 
        print_except=print_except,
        tqdm_silent=True,
        use_chat=True,
        model='gpt-3.5-turbo-0301',
    )
    extracted = records['messages'][0][-1]['content']
    extracted = extracted.replace(' ', '')
    try:
        if 'None' in extracted:
            raise ValueError('Invalid answer: %s' % message)
        extracted = type(extracted)
    except Exception as e:
        if print_except: 
            raise e
        extracted = None
    return extracted

In [ ]:
def extract(answers):
    prompts = [prompt(x) for x in answers]
    records = run_one_session(prompts, n_instances=1, print_except=False, tqdm_silent=True)
    extracted = records['messages'][0][-1]['content']
    return extracted

## Load Data

In [ ]:
# records = json.load(open('records/ultimatum_21_2023_03_29-05_41_19_PM.json', 'r'))
# records = json.load(open('records/trust_turbo_21_2023_04_12-04_47_23_PM.json', 'r'))
# records = json.load(open('records/trust_turbo_31_2023_04_12-04_53_06_PM.json', 'r'))
# records = json.load(open('records/trust_turbo_41_2023_04_12-04_58_06_PM.json', 'r'))
# records = json.load(open('records/trust_turbo_12_2023_04_12-04_32_40_PM.json', 'r'))
# records = json.load(open('records/trust_turbo_13_2023_04_12-04_37_21_PM.json', 'r'))
# records = json.load(open('records/trust_turbo_14_2023_04_12-04_41_54_PM.json', 'r'))
# records = json.load(open('records/trust_3_turbo-0613_2023_06_14-12_13_00_PM.json', 'r'))
# records = json.load(open('records/trust_3_gpt4-0613_2023_06_14-12_27_16_PM.json', 'r'))
# records = json.load(open('records/ultimatum_1_wo_ex_2023_03_13-11_54_45_PM.json', 'r'))
records = json.load(open('records/ultimatum_2_gpt4_2023_12_29-10_42_42_PM.json', 'r'))
answers = [inst[-1]['content'] for inst in records['messages']]
print(len(answers))

In [ ]:
answers

### Load Occupations

In [ ]:
# records_all, _ = json.load(open('records/dictator_turbo_occupations_described_2023_04_20-05_06_47_PM.json', 'r'))
# records_all, _ = json.load(open('records/ultimatum_1_turbo_occupations_described_2023_04_20-05_22_57_PM.json', 'r'))
# records_all, _ = json.load(open('records/ultimatum_2_turbo_occupations_described_2023_04_20-05_39_57_PM.json', 'r'))
# records_all, _ = json.load(open('records/trust_1_turbo_occupations_described_2023_04_20-05_51_24_PM.json', 'r'))
# records_all, _ = json.load(open('records/trust_2_turbo_occupations_described_2023_04_20-06_24_19_PM.json', 'r'))
records_all, _ = json.load(open('records/trust_3_turbo_occupations_described_2023_04_20-06_39_43_PM.json', 'r'))
# records_all, _ = json.load(open('records/trust_4_turbo_occupations_described_2023_04_20-07_18_05_PM.json', 'r'))
# records_all, _ = json.load(open('records/trust_3_turbo_0613_occupations_described_2023_06_14-12_47_10_PM.json', 'r'))
answers_all = {}
for occupation in records_all:
    answers_all[occupation] = [inst[-1]['content'] for inst in records_all[occupation]['messages']]
    print(occupation, len(answers_all[occupation]))

## Prompts

### Dictator, Ultimatum Proposer

In [ ]:
prompt = lambda x: '''
Below is a response of a game player, who proposed to give an amount of money to another player. What's the amount this player proposed to give *the other player*? Please output only the amount. If there is no answer or the answer is not claer, please output None.

Example:

Input: "Sure, here's my division:\n\nI would save $80 for myself and the other player gets $20. "

Output: 20

Input: "As an AI language model, I don\'t have the ability to hold a monetary value preference like humans. "

Output: None

Input: "As an AI language model, I don't have personal preferences or financial interests, and I can't make decisions for anyone. However, in a game theory context, a rational Proposal amount would depend on various factors such as the perceived value of the Responder's acceptance, the risk of rejection, and the motivation of maximizing monetary rewards. \n\nBased on these assumptions, a possible Proposal amount that may be attractive to the Responder and also considerate of the Proposer's interests could be [$60]. Another possible Proposal amount could be [$50]. However, this choice may differ depending on the individual's goals and the game's context. "

Output: None

Input: "{}"

Output: 
'''.format(x)

### Ultimatum Responder

In [ ]:
prompt = lambda x: '''
Below is a response of a game player, who was willing to accept an minimum amount of money from another player. What's the amount this player was willing to accept from *the other player*? Please output only the amount. If there is no answer or the answer is not claer, please output None.

Example:

Input: "As a mathematician, I would approach this game using mathematical principles such as game theory to determine the optimal decision. In this particular case, a rational approach would be for me to accept any proposal that offers me more than $0, as that is the only way that I can earn anything from the game. Therefore, I would accept any proposal that offers me a positive payoff, even if it is only a very small amount. For example, if the proposer proposed to give me $1 and keep $99 for themselves, I would accept it, and my response would be: \"I accept the proposal of [$1] for me and [$99] for the proposer.\""

Output: 1

Input: "As the Responder, the minimal amount in the proposal that I would accept depends on the proposal made by the Proposer. If the Proposer makes a fair proposal (50-50 division of $100), I would accept the proposal as it is the most equitable division of the money. However, if the Proposer makes an unfair proposal, my minimal acceptable amount would depend on the degree of unfairness. For example, if the proposal is for me to receive only 10% of the money, I may choose to reject the proposal. \n\nTherefore, I cannot give a concrete choice unless I know the Proposal being made."

Input: "{}"

Output: 
'''.format(x)

### Trust 1 (Investor)

In [ ]:
prompt = lambda x: '''
Below is a response of a game player, who proposed to invest an amount of money to another player. What's the amount this player proposed to invest *the other player*? Please output only the amount. If there is no answer or the answer is not claer, please output None.

Example:

Input: "As an Investor, I would invest [$50] with the Banker. This investment would grow to $150 under the Banker's 3x growth policy. Even though the Banker has control over how much to return to me, by choosing to invest $50, I am still able to retain a significant portion of the profits if the Banker decides to return less than the total amount. Additionally, investing half of my total amount provides a good balance of risk and potential profit.
"

Output: 50

Input: "{}"

Output: 
'''.format(x)

### Trust 2,3,4 (Banker)

In [ ]:
prompt = lambda x: '''
Below is a response of a game player, who proposed to return an amount of money to another player. What's the amount this player proposed to return *the other player*? Please output only the amount. If there is no answer or the answer is not claer, please output None.

Example:

Input: " 'As a Banker, I would like to return [$24] to the Investor. \n\nI want to maintain a balance between keeping a reasonable profit for myself and returning a good amount to the Investor. Returning the full amount of $30 would be generous, but it would leave me with no profit. On the other hand, returning a very small amount such as $3 would not be fair to the Investor who would lose a significant portion of their investment. Therefore, I feel that returning $24 is a reasonable choice because it provides a good balance between making a profit and ensuring that the Investor is happy with their return on investment. This would keep $6 as my profit, which is a 100% return on the investment of the Investor."

Output: 24

Input: "{}"

Output: 
'''.format(x)

In [ ]:
answer = '''
This is known as the ultimatum game. As the Responder, my choice would depend on the insightful negotiation skills of the Proposer, but generally, I would accept a proposal of [$30] or higher. This is because I would want to earn a reasonable amount of money from the game, but I also want to signal to the Proposer that I require an equitable proposal.
'''

In [ ]:
extract_chatgpt(answer, prompt, print_except=True)

## Extract

In [ ]:
extracted_list = []
for answer in tqdm(answers):
    extracted = extract([answer])
    extracted_list.append(extracted)

In [ ]:
print(', '.join(extracted_list))

### Extract Occupations

In [ ]:
to_print = []
for occupation in answers_all:
    print(occupation)
    answers = answers_all[occupation]
    extracted_list = []
    for answer in tqdm(answers):
        try:
            extracted = extract_chatgpt(answer, prompt, print_except=True)
        except Exception as e:
            print(e)
            continue
            # extracted = None
        extracted_list.append(str(extracted))
    to_print.append(', '.join(extracted_list))
print('\n'.join(to_print))